# Foundation of Computer Science

Richiesta:

1.Normalize the loan_lenders table. In the normalized table, each row must have one loan_id and one lender. 

2.For each loan, add a column duration corresponding to the number of days between the disburse time and the planned expiration time. If any of those two dates is missing, also the duration must be missing.

3.Find the lenders that have funded at least twice.

4.For each country, compute how many loans have involved that country as borrowers.

5.For each country, compute the overall amount of money borrowed.

6.Like the previous point, but expressed as a percentage of the overall amount lent.

7.Like the three previous points, but split for each year (with respect to disburse time).

8.For each lender, compute the overall amount of money lent. For each loan that has more than one lender, you must assume that all lenders contributed the same amount.

9.For each country, compute the difference between the overall amount of money lent and the overall amount of money borrowed. Since the country of the lender is often unknown, you can assume that the true distribution among the countries is the same as the one computed from the rows where the country is known.

10.Which country has the highest ratio between the difference computed at the previous point and the population?

11.Which country has the highest ratio between the difference computed at point 9 and the population that is not below the poverty line?

12.For each year, compute the total amount of loans. Each loan that has planned expiration time and disburse time in different years must have its amount distributed proportionally to the number of days in each year. For example, a loan with disburse time December 1st, 2016, planned expiration time January 30th 2018, and amount 5000USD has an amount of 5000USD 31 / (31+365+30) = 363.85 for 2016, 5000USD 365 / (31+365+30) = 4284.04 for 2017, and 5000USD * 30 / (31+365+30) = 352.11 for 2018.

In [2]:
import pandas as pd
import numpy as np
import datetime as dt

#### 1.Normalize the loan_lenders table. In the normalized table, each row must have one loan_id and one lender. 

In [3]:
loans_lenders=pd.read_csv('loans_lenders.csv')
loans_lenders.head()

,loan_id,lenders
0,483693,"muc888, sam4326, camaran3922, lachheb1865, reb..."
1,483738,"muc888, nora3555, williammanashi, barbara5610,..."
2,485000,"muc888, terrystl, richardandsusan8352, sherri4..."
3,486087,"muc888, james5068, rudi5955, daniel9859, don92..."
4,534428,"muc888, niki3008, teresa9174, mike4896, david7..."


In [4]:
norm=loans_lenders.assign(lenders=loans_lenders['lenders'].str.split(',')).explode('lenders')

In [5]:
norm.reset_index(drop= True)

,loan_id,lenders
0,483693,muc888
1,483693,sam4326
2,483693,camaran3922
3,483693,lachheb1865
4,483693,rebecca3499
...,...,...
28293926,1206425,trogdorfamily7622
28293927,1206425,danny6470
28293928,1206425,don6118
28293929,1206486,alan5175


#### 2.For each loan, add a column duration corresponding to the number of days between the disburse time and the planned expiration time. If any of those two dates is missing, also the duration must be missing.

In [98]:
loans= pd.read_csv('loans.csv')

In [22]:
loans.head()

,loan_id,loan_name,original_language,description,description_translated,funded_amount,loan_amount,status,activity_name,sector_name,...,raised_time,lender_term,num_lenders_total,num_journal_entries,num_bulk_entries,tags,borrower_genders,borrower_pictured,repayment_interval,distribution_model
0,657307,Aivy,English,"Aivy, 21 years of age, is single and lives in ...",NaN,125.0,125.0,funded,General Store,Retail,...,2014-01-15 04:48:22.000 +0000,7.0,3,2,1,NaN,female,true,irregular,field_partner
1,657259,Idalia Marizza,Spanish,"Doña Idalia, esta casada, tiene 57 años de eda...","Idalia, 57, is married and lives with her husb...",400.0,400.0,funded,Used Clothing,Clothing,...,2014-02-25 06:42:06.000 +0000,8.0,11,2,1,NaN,female,true,monthly,field_partner
2,658010,Aasia,English,Aasia is a 45-year-old married lady and she ha...,NaN,400.0,400.0,funded,General Store,Retail,...,2014-01-24 23:06:18.000 +0000,14.0,16,2,1,"#Woman Owned Biz, #Supporting Family, user_fav...",female,true,monthly,field_partner
3,659347,Gulmira,Russian,"Гулмире 36 лет, замужем, вместе с супругом вос...",Gulmira is 36 years old and married. She and ...,625.0,625.0,funded,Farming,Agriculture,...,2014-01-22 05:29:28.000 +0000,14.0,21,2,1,user_favorite,female,true,monthly,field_partner
4,656933,Ricky\t,English,Ricky is a farmer who currently cultivates his...,NaN,425.0,425.0,funded,Farming,Agriculture,...,2014-01-14 17:29:27.000 +0000,7.0,15,2,1,"#Animals, #Eco-friendly, #Sustainable Ag",male,true,bullet,field_partner


In [7]:
loans['disburse_time']=pd.to_datetime(loans['disburse_time'])
loans['planned_expiration_time']=pd.to_datetime(loans['planned_expiration_time'])
loans['duration']= loans['planned_expiration_time']-loans['disburse_time']
#eventuali na generano risultati na in automatico

In [28]:
loans[['loan_id', 'planned_expiration_time', 'disburse_time', 'duration']]

,loan_id,planned_expiration_time,disburse_time,duration
0,657307,2014-02-14 03:30:06+00:00,2013-12-22 08:00:00+00:00,53 days 19:30:06
1,657259,2014-03-26 22:25:07+00:00,2013-12-20 08:00:00+00:00,96 days 14:25:07
2,658010,2014-02-15 21:10:05+00:00,2014-01-09 08:00:00+00:00,37 days 13:10:05
3,659347,2014-02-21 03:10:02+00:00,2014-01-17 08:00:00+00:00,34 days 19:10:02
4,656933,2014-02-13 06:10:02+00:00,2013-12-17 08:00:00+00:00,57 days 22:10:02
...,...,...,...,...
1419602,988180,2016-01-02 01:00:03+00:00,2015-11-23 08:00:00+00:00,39 days 17:00:03
1419603,988213,2016-01-02 16:40:07+00:00,2015-11-24 08:00:00+00:00,39 days 08:40:07
1419604,989109,2016-01-03 22:20:04+00:00,2015-11-13 08:00:00+00:00,51 days 14:20:04
1419605,989143,2016-01-05 08:50:02+00:00,2015-11-03 08:00:00+00:00,63 days 00:50:02


#### 3.Find the lenders that have funded at least twice.

In [29]:
lenders_times = norm.groupby('lenders').count()
lenders_times.head()

,loan_id
lenders,
000,39
00000,39
0002,70
00mike00,1
0101craign0101,71


In [33]:
lenders_times.columns=['count']

In [34]:
lenders_two_times= lenders_times[lenders_times['count']>1]
lenders_two_times

,count
lenders,
000,39
00000,39
0002,70
0101craign0101,71
0132575,4
...,...
zyrorl,3
zzaman,11
zzanita,2


#### 4.For each country, compute how many loans have involved that country as borrowers

In [35]:
loans_country = loans.groupby('country_name').count()
loans_country

,loan_id,loan_name,original_language,description,description_translated,funded_amount,loan_amount,status,activity_name,sector_name,...,lender_term,num_lenders_total,num_journal_entries,num_bulk_entries,tags,borrower_genders,borrower_pictured,repayment_interval,distribution_model,duration
country_name,,,,,,,,,,,,,,,,,,,,,
Afghanistan,2337,1684,1684,1684,2,2337,2337,2337,2337,2337,...,2337,2337,2337,2337,12,1684,1684,2337,2337,2
Albania,3075,2998,2998,2998,2162,3075,3075,3075,3075,3075,...,3075,3075,3075,3075,2379,2998,2998,3075,3075,3075
Armenia,13952,13926,13926,13926,9568,13952,13952,13952,13952,13952,...,13952,13952,13952,13952,10622,13926,13926,13952,13952,12809
Azerbaijan,10172,9890,9890,9890,2320,10172,10172,10172,10172,10172,...,10172,10172,10172,10172,2576,9890,9890,10172,10172,4026
Belize,218,171,171,171,92,218,218,218,218,218,...,218,218,218,218,83,171,171,218,218,218
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Vietnam,21839,21788,21788,21767,13715,21839,21839,21839,21839,21839,...,21839,21839,21839,21839,12417,21788,21788,21839,21839,16371
Virgin Islands,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,0,2,2,2,2,0
Yemen,4206,3436,3436,3436,1508,4206,4206,4206,4206,4206,...,4206,4206,4206,4206,2224,3436,3436,4206,4206,4184


In [40]:
loans_country = loans_country[['loan_id']]
loans_country.columns=['times']
loans_country

,times
country_name,
Afghanistan,2337
Albania,3075
Armenia,13952
Azerbaijan,10172
Belize,218
...,...
Vietnam,21839
Virgin Islands,2
Yemen,4206


#### 5.For each country, compute the overall amount of money borrowed

In [18]:
money_country = loans.groupby('country_name')['loan_amount'].sum()

In [19]:
money_country=money_country.to_frame()
money_country

,loan_amount
country_name,
Afghanistan,1967950.0
Albania,4307350.0
Armenia,22950475.0
Azerbaijan,14784625.0
Belize,150175.0
...,...
Vietnam,24681100.0
Virgin Islands,10000.0
Yemen,3444000.0


#### 6.Like the previous point, but expressed as a percentage of the overall amount lent.

In [57]:
money_country['percentage']= money_country['loan_amount']/sum(money_country['loan_amount'])*100

In [58]:
money_country

,loan_amount,percentage
country_name,,
Afghanistan,1967950.0,0.166573
Albania,4307350.0,0.364586
Armenia,22950475.0,1.942589
Azerbaijan,14784625.0,1.251410
Belize,150175.0,0.012711
...,...,...
Vietnam,24681100.0,2.089074
Virgin Islands,10000.0,0.000846
Yemen,3444000.0,0.291509


#### 7.Like the three previous points, but split for each year (with respect to disburse time).

In [61]:
loans['disburse_year']= loans['disburse_time'].dt.year
loans[['disburse_year', 'disburse_time']]

,disburse_year,disburse_time
0,2013.0,2013-12-22 08:00:00+00:00
1,2013.0,2013-12-20 08:00:00+00:00
2,2014.0,2014-01-09 08:00:00+00:00
3,2014.0,2014-01-17 08:00:00+00:00
4,2013.0,2013-12-17 08:00:00+00:00
...,...,...
1419602,2015.0,2015-11-23 08:00:00+00:00
1419603,2015.0,2015-11-24 08:00:00+00:00
1419604,2015.0,2015-11-13 08:00:00+00:00
1419605,2015.0,2015-11-03 08:00:00+00:00


like point 4

In [63]:
loans_country_year = loans.groupby(['country_name', 'disburse_year'] ).count()['loan_id']
loans_country_year= loans_country_year.to_frame()
loans_country_year

loan_id
country_name disburse_year         
Afghanistan  2007.0             408
             2008.0             370
             2009.0             678
             2010.0             632
             2011.0             247
...                             ...
Zimbabwe     2013.0             426
             2014.0            2078
             2015.0             600
             2016.0             808
             2017.0            1079

[748 rows x 1 columns]

In [64]:
loans_country_year.columns=['times']
loans_country_year

times
country_name disburse_year       
Afghanistan  2007.0           408
             2008.0           370
             2009.0           678
             2010.0           632
             2011.0           247
...                           ...
Zimbabwe     2013.0           426
             2014.0          2078
             2015.0           600
             2016.0           808
             2017.0          1079

[748 rows x 1 columns]

like point 5

In [65]:
money_country_year = loans.groupby(['country_name', 'disburse_year']).sum()['loan_amount']
money_country_year=money_country_year.to_frame()
money_country_year

loan_amount
country_name disburse_year             
Afghanistan  2007.0            194975.0
             2008.0            365375.0
             2009.0            585125.0
             2010.0            563350.0
             2011.0            245125.0
...                                 ...
Zimbabwe     2013.0            678525.0
             2014.0           1311575.0
             2015.0            723625.0
             2016.0            788600.0
             2017.0           1237600.0

[748 rows x 1 columns]

like point 6

In [66]:
money_country_year['percentage']= money_country_year['loan_amount']/sum(money_country_year['loan_amount'])*100
money_country_year

loan_amount  percentage
country_name disburse_year                         
Afghanistan  2007.0            194975.0    0.016657
             2008.0            365375.0    0.031215
             2009.0            585125.0    0.049989
             2010.0            563350.0    0.048129
             2011.0            245125.0    0.020942
...                                 ...         ...
Zimbabwe     2013.0            678525.0    0.057969
             2014.0           1311575.0    0.112053
             2015.0            723625.0    0.061822
             2016.0            788600.0    0.067373
             2017.0           1237600.0    0.105733

[748 rows x 2 columns]

#### 8.For each lender, compute the overall amount of money lent. For each loan that has more than one lender, you must assume that all lenders contributed the same amount.

In [8]:
lenders_per_loan = norm.groupby('loan_id').count()
lenders_per_loan

,lenders
loan_id,
84,3
85,2
86,3
88,3
89,4
...,...
1444051,1
1444053,1
1444058,1


In [9]:
lenders_per_loan.columns=['num_lenders']
lenders_per_loan

,num_lenders
loan_id,
84,3
85,2
86,3
88,3
89,4
...,...
1444051,1
1444053,1
1444058,1


In [12]:
money_lent = lenders_per_loan.merge(loans[['loan_id', 'loan_amount']], on='loan_id')
money_lent

,loan_id,num_lenders,loan_amount
0,84,3,500.0
1,85,2,500.0
2,86,3,500.0
3,88,3,300.0
4,89,4,500.0
...,...,...,...
1387423,1444051,1,225.0
1387424,1444053,1,725.0
1387425,1444058,1,25.0
1387426,1444063,1,50.0


In [13]:
money_lent['money_per_lender']=money_lent['loan_amount']/money_lent['num_lenders']
money_lent

,loan_id,num_lenders,loan_amount,money_per_lender
0,84,3,500.0,166.666667
1,85,2,500.0,250.000000
2,86,3,500.0,166.666667
3,88,3,300.0,100.000000
4,89,4,500.0,125.000000
...,...,...,...,...
1387423,1444051,1,225.0,225.000000
1387424,1444053,1,725.0,725.000000
1387425,1444058,1,25.0,25.000000
1387426,1444063,1,50.0,50.000000


In [14]:
norm_money = norm.merge(money_lent[['loan_id', 'money_per_lender']], on='loan_id')
norm_money

,loan_id,lenders,money_per_lender
0,483693,muc888,30.625
1,483693,sam4326,30.625
2,483693,camaran3922,30.625
3,483693,lachheb1865,30.625
4,483693,rebecca3499,30.625
...,...,...,...
28293907,1206425,trogdorfamily7622,40.625
28293908,1206425,danny6470,40.625
28293909,1206425,don6118,40.625
28293910,1206486,alan5175,75.000


In [15]:
norm_money=norm_money.groupby('lenders').sum()['money_per_lender']
norm_money

lenders
 000               1672.618411
 00000             1380.693644
 0002              2472.563566
 00mike00            52.631579
 0101craign0101    2623.565117
                      ...     
zzanita              87.500000
zzcyna7269           55.357143
zzinnia              38.000000
zzmcfate           2287.291955
zzrvmf8538          106.417625
Name: money_per_lender, Length: 1639026, dtype: float64

#### 9.For each country, compute the difference between the overall amount of money lent and the overall amount of money borrowed. Since the country of the lender is often unknown, you can assume that the true distribution among the countries is the same as the one computed from the rows where the country is known.

In [20]:
money_funded=loans.groupby('country_name')['funded_amount'].sum()
money_funded=money_funded.to_frame()
money_funded

,funded_amount
country_name,
Afghanistan,1964650.0
Albania,3992475.0
Armenia,20284675.0
Azerbaijan,14191350.0
Belize,150175.0
...,...
Vietnam,23463825.0
Virgin Islands,0.0
Yemen,3340025.0


In [21]:
differences= money_country.merge(money_funded, how= 'outer', on= 'country_name')
differences['diff']= differences['loan_amount']- differences['funded_amount']

In [22]:
differences

,loan_amount,funded_amount,diff
country_name,,,
Afghanistan,1967950.0,1964650.0,3300.0
Albania,4307350.0,3992475.0,314875.0
Armenia,22950475.0,20284675.0,2665800.0
Azerbaijan,14784625.0,14191350.0,593275.0
Belize,150175.0,150175.0,0.0
...,...,...,...
Vietnam,24681100.0,23463825.0,1217275.0
Virgin Islands,10000.0,0.0,10000.0
Yemen,3444000.0,3340025.0,103975.0


#### 10.Which country has the highest ratio between the difference computed at the previous point and the population?

In [23]:
country= pd.read_csv('country_stats.csv')

In [24]:
country

,country_name,country_code,country_code3,continent,region,population,population_below_poverty_line,hdi,life_expectancy,expected_years_of_schooling,mean_years_of_schooling,gni,kiva_country_name
0,India,IN,IND,Asia,Southern Asia,1339180127,21.9,0.623559,68.322,11.696590,6.298834,5663.474799,India
1,Nigeria,NG,NGA,Africa,Western Africa,190886311,70.0,0.527105,53.057,9.970482,6.000000,5442.901264,Nigeria
2,Mexico,MX,MEX,Americas,Central America,129163276,46.2,0.761683,76.972,13.299090,8.554985,16383.106680,Mexico
3,Pakistan,PK,PAK,Asia,Southern Asia,197015955,29.5,0.550354,66.365,8.106910,5.089460,5031.173074,Pakistan
4,Bangladesh,BD,BGD,Asia,Southern Asia,164669751,31.5,0.578824,71.985,10.178706,5.241577,3341.490722,Bangladesh
...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,Somalia,SO,SOM,Africa,Eastern Africa,14742523,NaN,NaN,NaN,NaN,NaN,NaN,Somalia
170,Central African Republic,CF,CAF,Africa,Middle Africa,4659080,NaN,0.352440,51.458,7.098980,4.230000,587.473961,Central African Republic
171,Samoa,WS,WSM,Oceania,Polynesia,196440,NaN,0.702000,NaN,NaN,NaN,NaN,Samoa
172,Palestine,PS,PS,Asia,Western Asia,4920724,NaN,0.677000,NaN,NaN,NaN,NaN,Palestine


In [26]:
country_money= differences.merge(country[['country_name', 'population', 'population_below_poverty_line']], on= 'country_name')
country_money

,country_name,loan_amount,funded_amount,diff,population,population_below_poverty_line
0,Afghanistan,1967950.0,1964650.0,3300.0,35530081,35.8
1,Albania,4307350.0,3992475.0,314875.0,2930187,14.3
2,Armenia,22950475.0,20284675.0,2665800.0,2930450,32.0
3,Azerbaijan,14784625.0,14191350.0,593275.0,9827589,4.9
4,Belize,150175.0,150175.0,0.0,374681,41.0
...,...,...,...,...,...,...
81,United States,46352000.0,36365340.0,9986660.0,324459463,15.1
82,Uruguay,8000.0,8000.0,0.0,3456750,9.7
83,Yemen,3444000.0,3340025.0,103975.0,28250420,54.0
84,Zambia,1978975.0,1913250.0,65725.0,17094130,60.5


In [27]:
country_money['ratio']= country_money['diff']/country_money['population']
country_money

,country_name,loan_amount,funded_amount,diff,population,population_below_poverty_line,ratio
0,Afghanistan,1967950.0,1964650.0,3300.0,35530081,35.8,0.000093
1,Albania,4307350.0,3992475.0,314875.0,2930187,14.3,0.107459
2,Armenia,22950475.0,20284675.0,2665800.0,2930450,32.0,0.909690
3,Azerbaijan,14784625.0,14191350.0,593275.0,9827589,4.9,0.060368
4,Belize,150175.0,150175.0,0.0,374681,41.0,0.000000
...,...,...,...,...,...,...,...
81,United States,46352000.0,36365340.0,9986660.0,324459463,15.1,0.030779
82,Uruguay,8000.0,8000.0,0.0,3456750,9.7,0.000000
83,Yemen,3444000.0,3340025.0,103975.0,28250420,54.0,0.003680
84,Zambia,1978975.0,1913250.0,65725.0,17094130,60.5,0.003845


In [30]:
country_money.sort_values("ratio", axis = 0, ascending = False)

,country_name,loan_amount,funded_amount,diff,population,population_below_poverty_line,ratio
64,Samoa,12212175.0,11508250.0,703925.0,196440,NaN,3.583410
2,Armenia,22950475.0,20284675.0,2665800.0,2930450,32.0,0.909690
26,El Salvador,41691550.0,37881675.0,3809875.0,6377853,34.9,0.597360
73,Tajikistan,39622125.0,37049825.0,2572300.0,8921343,31.5,0.288331
56,Palestine,22316675.0,20911075.0,1405600.0,4920724,NaN,0.285649
...,...,...,...,...,...,...,...
41,Lesotho,516450.0,516450.0,0.0,2233339,57.0,0.000000
46,Mauritania,15000.0,15000.0,0.0,4420184,31.0,0.000000
51,Namibia,39525.0,39525.0,0.0,2533794,28.7,0.000000
71,Sri Lanka,74800.0,74800.0,0.0,20876917,6.7,0.000000


#### 11.Which country has the highest ratio between the difference computed at point 9 and the population that is not below the poverty line?

In [34]:
country_money['poor_pop']= country_money['population']* country_money['population_below_poverty_line']/100

In [35]:
country_money['ratio2']= country_money['diff']/country_money['poor_pop']
country_money.sort_values("ratio2", axis = 0, ascending = False)

,country_name,loan_amount,funded_amount,diff,population,population_below_poverty_line,ratio,poor_pop,ratio2
2,Armenia,22950475.0,20284675.0,2665800.0,2930450,32.0,0.909690,937744.000,2.842780
26,El Salvador,41691550.0,37881675.0,3809875.0,6377853,34.9,0.597360,2225870.697,1.711634
27,Georgia,9570425.0,8994450.0,575975.0,3912061,9.2,0.147231,359909.612,1.600332
3,Azerbaijan,14784625.0,14191350.0,593275.0,9827589,4.9,0.060368,481551.861,1.232006
73,Tajikistan,39622125.0,37049825.0,2572300.0,8921343,31.5,0.288331,2810223.045,0.915337
...,...,...,...,...,...,...,...,...,...
56,Palestine,22316675.0,20911075.0,1405600.0,4920724,NaN,0.285649,NaN,NaN
62,Puerto Rico,441900.0,362600.0,79300.0,3663131,NaN,0.021648,NaN,NaN
64,Samoa,12212175.0,11508250.0,703925.0,196440,NaN,3.583410,NaN,NaN
67,Solomon Islands,815575.0,806275.0,9300.0,611343,NaN,0.015212,NaN,NaN


#### 12.For each year, compute the total amount of loans. Each loan that has planned expiration time and disburse time in different years must have its amount distributed proportionally to the number of days in each year. For example, a loan with disburse time December 1st, 2016, planned expiration time January 30th 2018, and amount 5000USD has an amount of 5000USD 31 / (31+365+30) = 363.85 for 2016, 5000USD 365 / (31+365+30) = 4284.04 for 2017, and 5000USD * 30 / (31+365+30) = 352.11 for 2018.

In [40]:
loans=loans.dropna(subset=['planned_expiration_time'])
loans=loans.dropna(subset=['disburse_time'])
loans=loans.reset_index(drop=True )
loans

,loan_id,loan_name,original_language,description,description_translated,funded_amount,loan_amount,status,activity_name,sector_name,...,lender_term,num_lenders_total,num_journal_entries,num_bulk_entries,tags,borrower_genders,borrower_pictured,repayment_interval,distribution_model,duration
0,657307,Aivy,English,"Aivy, 21 years of age, is single and lives in ...",NaN,125.0,125.0,funded,General Store,Retail,...,7.0,3,2,1,NaN,female,true,irregular,field_partner,53 days 19:30:06
1,657259,Idalia Marizza,Spanish,"Doña Idalia, esta casada, tiene 57 años de eda...","Idalia, 57, is married and lives with her husb...",400.0,400.0,funded,Used Clothing,Clothing,...,8.0,11,2,1,NaN,female,true,monthly,field_partner,96 days 14:25:07
2,658010,Aasia,English,Aasia is a 45-year-old married lady and she ha...,NaN,400.0,400.0,funded,General Store,Retail,...,14.0,16,2,1,"#Woman Owned Biz, #Supporting Family, user_fav...",female,true,monthly,field_partner,37 days 13:10:05
3,659347,Gulmira,Russian,"Гулмире 36 лет, замужем, вместе с супругом вос...",Gulmira is 36 years old and married. She and ...,625.0,625.0,funded,Farming,Agriculture,...,14.0,21,2,1,user_favorite,female,true,monthly,field_partner,34 days 19:10:02
4,656933,Ricky\t,English,Ricky is a farmer who currently cultivates his...,NaN,425.0,425.0,funded,Farming,Agriculture,...,7.0,15,2,1,"#Animals, #Eco-friendly, #Sustainable Ag",male,true,bullet,field_partner,57 days 22:10:02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1044957,988180,NaN,NaN,NaN,NaN,400.0,400.0,funded,Tailoring,Services,...,14.0,16,4,2,"#Parent, #Repeat Borrower, #Woman Owned Biz",NaN,NaN,monthly,field_partner,39 days 17:00:03
1044958,988213,Perlita,English,"Perlita is 52 years old, married and has three...","Perlita is 52 years old, married and has three...",300.0,300.0,funded,Pigs,Agriculture,...,14.0,12,1,1,"#Animals, #Elderly, #Repeat Borrower, #Woman O...",female,true,irregular,field_partner,39 days 08:40:07
1044959,989109,Okyeso Nyame Group,English,Okyeso Nyame group will begin its third cycle ...,Okyeso Nyame group will begin its third cycle ...,2425.0,2425.0,funded,Bakery,Food,...,8.0,76,2,1,"user_favorite, #Parent, #Vegan, #Woman Owned B...","female, female, female, male, male, female","true, true, true, true, true, true",irregular,field_partner,51 days 14:20:04
1044960,989143,Exequila,English,"Exequila is from San Miguel, Bohol. She is in...","Exequila is from San Miguel, Bohol. She is in...",100.0,100.0,funded,Farming,Agriculture,...,12.0,3,1,1,NaN,female,true,irregular,field_partner,63 days 00:50:02


In [41]:
#prestito piu vecchio
loans.iloc[loans['disburse_time'].idxmin()]

loan_id                                                                       358219
loan_name                                                                 Mirismayil
original_language                                                            English
description                        Mirismayil is a 28-year-old trader who sells d...
description_translated                                                           NaN
funded_amount                                                                   2300
loan_amount                                                                     2300
status                                                                        funded
activity_name                                                         Clothing Sales
sector_name                                                                 Clothing
loan_use                                                    to purchase more clothes
country_code                                                     

In [42]:
#prestito piu recente
loans.iloc[loans['planned_expiration_time'].idxmax()]

loan_id                                                                      1444085
loan_name                                                                        NaN
original_language                                                            English
description                        To pay for feed and a hutch for the turduckens...
description_translated             To pay for feed and a hutch for the turduckens...
funded_amount                                                                      0
loan_amount                                                                       25
status                                                                   fundRaising
activity_name                                                              Livestock
sector_name                                                              Agriculture
loan_use                           to start a turducken farm. - this loan use has...
country_code                                                     

In [46]:
#ammontare giornaliero
loans['daily']= loans['loan_amount']/loans['duration'].dt.days
loans['daily']

0           2.358491
1           4.166667
2          10.810811
3          18.382353
4           7.456140
             ...    
1044957    10.256410
1044958     7.692308
1044959    47.549020
1044960     1.587302
1044961     2.868852
Name: daily, Length: 1044962, dtype: float64

In [92]:
diz={}
for current_year in range(2011, 2019): #per ogni anno nel range dei prestiti
    temp=loans[(loans['disburse_time'].dt.year<=current_year) & (loans['planned_expiration_time'].dt.year>=current_year)]
    diz[current_year]=0 #selezioni i prestiti in corso in quell anno
    for row in temp.index:
        if temp.at[row, 'disburse_time'].year < current_year: #iniziato prima
            if temp.at[row, 'planned_expiration_time'].year == current_year: #finisce nell anno
                diz[current_year]+= temp.at[row, 'planned_expiration_time'].dayofyear* temp.at[row, 'daily']
            if temp.at[row, 'planned_expiration_time'].year > current_year: #non finisce in quell anno
                if (current_year == 2012 | current_year == 2016):
                    diz[current_year]+= 366* temp.at[row, 'daily']
                else:
                    diz[current_year]+= 365* temp.at[row, 'daily']
        if temp.at[row, 'disburse_time'].year == current_year: #iniziato in quell anno
            if temp.at[row, 'planned_expiration_time'].year == current_year: #finisce nell anno
                diz[current_year]+= temp.at[row, 'loan_amount']
            if temp.at[row, 'planned_expiration_time'].year > current_year: #non finisce in quell anno
                if (current_year == 2012 | current_year == 2016):
                    diz[current_year]+= (366-temp.at[row, 'disburse_time'].dayofyear) * temp.at[row, 'daily']
                else:
                    diz[current_year]+= (365-temp.at[row, 'disburse_time'].dayofyear) * temp.at[row, 'daily']


In [93]:
diz

{2011: 631678.3440311765,
 2012: 113022011.54589057,
 2013: 125036380.29227056,
 2014: 153278613.4132631,
 2015: 159882303.38875186,
 2016: 158649804.14773685,
 2017: 171979395.62687504,
 2018: 5735473.345945817}